In [1]:
import numpy as np
import networkx as nx

#%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
%config InlineBackend.figure_format = 'retina'

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader # (testset, batch_size=4,shuffle=False, num_workers=4)
from torch.optim.lr_scheduler import ReduceLROnPlateau as RLRP
from torch.nn.parallel import DistributedDataParallel, DataParallel
from torch.nn.init import xavier_normal
from torch.nn.parameter import Parameter
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import sys
from datetime import datetime
from functools import reduce
import os
import os.path
import pandas as pd
import pickle
import importlib
from collections import Counter
from copy import deepcopy
from collections import OrderedDict

import torch_geometric as tg
import nkmodel as nk
import ppo.core as core
from ppo.ppo import PPOBuffer
import envs
import json
from utils.utils import max_mean_clustering_network

In [2]:
import envs
import ppo.core as core
from utils.utils import DataGen, AverageMeter
from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
import ppo.net as net

In [92]:
E = 32
M = 100
N = 15
K = 7
NN = 3
exp = 8
trj_len = 200
graph_type = 'complete'
reward_type = 'indv_raw_full'
action_type = 'total'
extra_type = 'SI'
env_name = 'SL_NK_' + action_type
ac_kwargs = {'activation': nn.Tanh()}
nx_dict = {'complete': nx.complete_graph, 'ba': nx.barabasi_albert_graph, 'er': nx.erdos_renyi_graph, 'maxmc':max_mean_clustering_network} 
nx_arg_dict = {'complete': {'n': M}, 'ba': {'n': M, 'm': 19}, 'er': {'n': M, 'p': 0.3}, 'maxmc': {'n': M}}

env_kwargs = {
        'E': E,
        'M': M,
        'N': N,
        'K': K,
        'neighbor_num': NN,
        'exp': exp,
        'graph': nx_dict[graph_type],
        'graph_dict': nx_arg_dict[graph_type],
        'reward_type': reward_type,
        'action_type': action_type,
        'extra_type': extra_type,
        'corr_type': 'TT'
    }

env = envs.__dict__[env_name](**env_kwargs) 

In [93]:
baseline_type = 'FollowBest'
exp_name = 'complete_total_FollowBest_SI_N15K7NN3'

batch_size = 16
batch_num = 2
total_size = batch_num * batch_size
train_ratio = 0.8

generator = DataGen(env, baseline_type, batch_size, batch_num)
if not os.path.isfile('./data/supervised/' + exp_name + '_train.pkl'):
    generator.run(exp_name, total_size, batch_size, train_ratio)

In [94]:
exp_name = 'complete_total_FollowBest_SI_N15K7NN3'
with open('./data/supervised/' + exp_name + '_train.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('./data/supervised/' + exp_name + '_test.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [95]:
train_data_image = np.concatenate(train_data['Image'], axis=0)
train_data_image = train_data_image.reshape(-1, *train_data_image.shape[-2:])
train_data_label = np.concatenate(train_data['Label'], axis=0)
train_data_label = train_data_label.reshape(-1, *train_data_label.shape[-1:])
test_data_image = np.concatenate(test_data['Image'], axis=0)
test_data_image = test_data_image.reshape(-1, *test_data_image.shape[-2:])
test_data_label = np.concatenate(test_data['Label'], axis=0)
test_data_label = test_data_label.reshape(-1, *test_data_label.shape[-1:])

train_data = TensorDataset(torch.FloatTensor(train_data_image), torch.FloatTensor(train_data_label))
test_data = TensorDataset(torch.FloatTensor(test_data_image), torch.FloatTensor(test_data_label))

train_loader = DataLoader(
            train_data,
            batch_size=batch_size,
            shuffle=True,
            pin_memory=True,
            num_workers=0
        )
test_loader = DataLoader(
            test_data,
            batch_size=batch_size,
            shuffle=True,
            pin_memory=True,
            num_workers=0
        )

In [48]:
def str2bool(v):
    return bool(strtobool(v))

In [49]:
from distutils.util import strtobool

In [88]:
model = net.__dict__['st']((4, 17), 30, **ac_kwargs)
exp_name = 'st_complete_total_FollowBest_SI_N15K7NN3_CE_sptest'
checkpoint = torch.load(f'./data/runs/{exp_name}/{exp_name}_s42/pyt_save/model.pth')
model.load_state_dict(checkpoint)
model = model.cuda()

In [77]:
x = OrderedDict((f'logits_net.{key}', value) for (key, value) in checkpoint.items())

In [76]:
checkpoint.keys()

odict_keys(['enc.0.I', 'enc.0.mab0.fc_q.weight', 'enc.0.mab0.fc_q.bias', 'enc.0.mab0.fc_k.weight', 'enc.0.mab0.fc_k.bias', 'enc.0.mab0.fc_v.weight', 'enc.0.mab0.fc_v.bias', 'enc.0.mab0.fc_o.weight', 'enc.0.mab0.fc_o.bias', 'enc.0.mab1.fc_q.weight', 'enc.0.mab1.fc_q.bias', 'enc.0.mab1.fc_k.weight', 'enc.0.mab1.fc_k.bias', 'enc.0.mab1.fc_v.weight', 'enc.0.mab1.fc_v.bias', 'enc.0.mab1.fc_o.weight', 'enc.0.mab1.fc_o.bias', 'enc.1.I', 'enc.1.mab0.fc_q.weight', 'enc.1.mab0.fc_q.bias', 'enc.1.mab0.fc_k.weight', 'enc.1.mab0.fc_k.bias', 'enc.1.mab0.fc_v.weight', 'enc.1.mab0.fc_v.bias', 'enc.1.mab0.fc_o.weight', 'enc.1.mab0.fc_o.bias', 'enc.1.mab1.fc_q.weight', 'enc.1.mab1.fc_q.bias', 'enc.1.mab1.fc_k.weight', 'enc.1.mab1.fc_k.bias', 'enc.1.mab1.fc_v.weight', 'enc.1.mab1.fc_v.bias', 'enc.1.mab1.fc_o.weight', 'enc.1.mab1.fc_o.bias', 'dec.0.S', 'dec.0.mab.fc_q.weight', 'dec.0.mab.fc_q.bias', 'dec.0.mab.fc_k.weight', 'dec.0.mab.fc_k.bias', 'dec.0.mab.fc_v.weight', 'dec.0.mab.fc_v.bias', 'dec.0.mab.

In [73]:
class test():
    def __init__(self, a, b=0, **kwargs):
        self.a = a
        self.b = b
    def show(self):
        print(self.a, self.b)

In [74]:
kwargs = {'b': 10}
y = test(5, **kwargs)
y.show()

5 10


In [96]:
for image, label in test_loader:
    break

In [97]:
i=5
print(image[i].long())
print(label[i].long())
pred = model(image.cuda()).reshape(-1, 2, 15)
print((pred[i][1]>0).long())

tensor([[0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 7, 0],
        [1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 5, 0],
        [1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0]])
tensor([0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0])
tensor([0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0], device='cuda:0')


In [99]:
i=7
print(image[i].long())
print(label[i].long())
pred = model(image.cuda()).reshape(-1, 2, 15)
print((pred[i][1]>0).long())

tensor([[1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1],
        [1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 7, 0],
        [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 3, 0]])
tensor([1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0])
tensor([1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0], device='cuda:0')


In [139]:
s1, s2, s3, s4 = 20, 30, 40, 20
template = np.array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, s1, 1], 
              [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, s2, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, s3, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, s4, 0]])
pred = model(torch.as_tensor(template, dtype=torch.float32, device='cuda').unsqueeze(0)).reshape(-1, 2, 15)
print((pred[0][1]>0).long())

tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0], device='cuda:0')


In [ ]:
d = len(train_data.tensors[0])
for i in range(d):
    image = train_data.tensors[0][i][0][:-len(extra_type)]
    score = 
    print(score)
        
print(c, d, c/d)

In [14]:
c = 0
d = len(train_data.tensors[0])
for i in range(d):
    image = train_data.tensors[0][i][0][:-len(extra_type)]
    label = train_data.tensors[1][i]
    if not torch.equal(image, label):
        c+=1
        
print(c, d, c/d)

1696 8200 0.20682926829268292


In [153]:
def test(model, test_loader, loss, criterion):
    test_losses = AverageMeter("TestLoss", ":.4e")
    for image, label in test_loader:
        label = label.cuda()
        image = image.cuda()
        pred = model(image)
        test_loss = loss(pred, label, criterion)
        test_losses.update(test_loss.item(), image.shape[0])
    return test_losses.avg

def CE(pred, label, criterion):
    pred = pred.view(pred.shape[0], -1, 2)
    return criterion(pred.transpose(-2, -1), label.long())

In [154]:
loss = CE
criterion = nn.CrossEntropyLoss()
    
x = test(model, test_loader, loss, criterion)